In [198]:
import pandas as pd
import numpy as np

# 1 - Retrieve initial model data

In [199]:
power_path = "../installed_capacity/generazione_domanda_per_zona_v02.xlsx"
data = { # from TYNDP_2024.ipynb
    2019:
    {
        'increase': 0, # %-increase of annual elc consumption from 2019
        'heating_share': 8, # share of heating over total elc consumption in %
        'cooling_share': 6, # share of cooling over total elc consumption in %
    },
    2040:
    {
        'increase': 28,
        'heating_share': 11,
        'cooling_share': 4,
    },
    2050:
    {
        'increase': 50,
        'heating_share': 9,
        'cooling_share': 4,
    },
}

print("============> Electricity profile: total (MW)")
df_power = pd.read_excel(power_path, sheet_name='Demand', index_col=0) # MW
display(df_power)
print(f"Total national demand (TWh): {round(df_power.sum().sum()/1e6,2)}\n")

zones = df_power.columns.tolist()
elc_demand_zonal = round(df_power.sum(), 2)
# Create demand dictionary with zones and populate it
demand = {}
for year in [2019, 2040, 2050]:
    demand[year] = {}
    for zone in zones:
        elc_demand_zone = elc_demand_zonal[zone]
        total_demand = elc_demand_zone * (1 + data[year]['increase']/100)
        heating_demand = total_demand * data[year]['heating_share']/100
        cooling_demand = total_demand * data[year]['cooling_share']/100
        demand[year][zone] = {
            'total': round(total_demand, 2),
            'heating': round(heating_demand, 2),
            'cooling': round(cooling_demand, 2)
        }

print("============> Heating and cooling fractions\n")
# Create dataframe from demand dictionary
rows = []
for year in demand:
    for zone in demand[year]:
        row = {
            'year': year,
            'zone': zone,
            'total': demand[year][zone]['total'],
            'heating': demand[year][zone]['heating'],
            'cooling': demand[year][zone]['cooling']
        }
        rows.append(row)

print("=====> Electricity yearly demand: (TWh)")
df_demand = pd.DataFrame(rows).set_index(['year', 'zone']).drop(columns=["total"]).sum(axis=1).unstack() # MWh
display(round(df_demand/1e6,2))

# Create profile dataframe with multiindex columns
profiles = {}
for year in [2019, 2040, 2050]:
    for zone in zones:
        # Base profile for the zone (in MW)
        base_profile = df_power[zone]
        
        # Apply increase percentage to get total profile
        total_profile = base_profile * (1 + data[year]['increase']/100)
        
        # Apply heating and cooling shares
        heating_profile = total_profile * data[year]['heating_share']/100
        cooling_profile = total_profile * data[year]['cooling_share']/100
        
        # Store in dictionary with tuple key
        profiles[(year, zone, 'total')] = total_profile
        profiles[(year, zone, 'heating')] = heating_profile
        profiles[(year, zone, 'cooling')] = cooling_profile

# Create dataframe with multiindex columns
print("============> Profile (MW)")
df_profile_initial = pd.DataFrame(profiles)
df_profile_initial.columns = pd.MultiIndex.from_tuples(df_profile_initial.columns, names=['year', 'zone', 'type'])
display(df_profile_initial)

============> Electricity profile: total (MW)


,CALA,CNOR,CSUD,NORD,SARD,SICI,SUD
hour,,,,,,,
1,557,2061,4830,11929,816,1867,1564
2,552,1980,4677,11576,778,1821,1381
3,494,1892,4405,11081,745,1695,1311
4,448,1794,4080,10667,721,1597,1284
5,437,1742,3916,10501,710,1540,1229
...,...,...,...,...,...,...,...
8756,794,2992,6304,17613,1223,2825,3238
8757,722,2802,5752,16194,1148,2620,3127
8758,638,2538,5342,14989,1056,2272,2856


Total national demand (TWh): 311.36

============> Heating and cooling fractions

=====> Electricity yearly demand: (TWh)


zone,CALA,CNOR,CSUD,NORD,SARD,SICI,SUD
year,,,,,,,
2019,0.84,3.68,7.41,24.55,1.18,2.66,3.27
2040,1.15,5.04,10.17,33.67,1.62,3.64,4.48
2050,1.17,5.12,10.33,34.20,1.65,3.70,4.55


============> Profile (MW)


year   2019                                                                  \
zone   CALA                    CNOR                    CSUD                   
type  total heating cooling   total heating cooling   total heating cooling   
hour                                                                          
1     557.0   44.56   33.42  2061.0  164.88  123.66  4830.0  386.40  289.80   
2     552.0   44.16   33.12  1980.0  158.40  118.80  4677.0  374.16  280.62   
3     494.0   39.52   29.64  1892.0  151.36  113.52  4405.0  352.40  264.30   
4     448.0   35.84   26.88  1794.0  143.52  107.64  4080.0  326.40  244.80   
5     437.0   34.96   26.22  1742.0  139.36  104.52  3916.0  313.28  234.96   
...     ...     ...     ...     ...     ...     ...     ...     ...     ...   
8756  794.0   63.52   47.64  2992.0  239.36  179.52  6304.0  504.32  378.24   
8757  722.0   57.76   43.32  2802.0  224.16  168.12  5752.0  460.16  345.12   
8758  638.0   51.04   38.28  2538.0  203.04  152.28  5342.0  427.36  320.52   
8759  590.0   47.20   35.40  2375.0  190.00  142.50  5162.0  412.96  309.72   
8760  559.0   44.72   33.54  2211.0  176.88  132.66  4824.0  385.92  289.44   

year           ...     2050                                                    \
zone     NORD  ...     NORD    SARD                     SICI                    
type    total  ...  cooling   total  heating cooling   total  heating cooling   
hour           ...                                                              
1     11929.0  ...   715.74  1224.0  110.160   48.96  2800.5  252.045  112.02   
2     11576.0  ...   694.56  1167.0  105.030   46.68  2731.5  245.835  109.26   
3     11081.0  ...   664.86  1117.5  100.575   44.70  2542.5  228.825  101.70   
4     10667.0  ...   640.02  1081.5   97.335   43.26  2395.5  215.595   95.82   
5     10501.0  ...   630.06  1065.0   95.850   42.60  2310.0  207.900   92.40   
...       ...  ...      ...     ...      ...     ...     ...      ...     ...   
8756  17613.0  ...  1056.78  1834.5  165.105   73.38  4237.5  381.375  169.50   
8757  16194.0  ...   971.64  1722.0  154.980   68.88  3930.0  353.700  157.20   
8758  14989.0  ...   899.34  1584.0  142.560   63.36  3408.0  306.720  136.32   
8759  14049.0  ...   842.94  1468.5  132.165   58.74  3159.0  284.310  126.36   
8760  13122.0  ...   787.32  1390.5  125.145   55.62  2905.5  261.495  116.22   

year                           
zone     SUD                   
type   total  heating cooling  
hour                           
1     2346.0  211.140   93.84  
2     2071.5  186.435   82.86  
3     1966.5  176.985   78.66  
4     1926.0  173.340   77.04  
5     1843.5  165.915   73.74  
...      ...      ...     ...  
8756  4857.0  437.130  194.28  
8757  4690.5  422.145  187.62  
8758  4284.0  385.560  171.36  
8759  3987.0  358.830  159.48  
8760  3880.5  349.245  155.22  

[8760 rows x 63 columns]

# 2- Retrieve ninja profiles

In [200]:
# Load ninja profiles for all zones:
# ninja locations (proxy of zones): NORD--> Milano, CNOR--> Bologna, CSUD--> Roma, SUD--> Napoli, CALA--> Calabria, SICI--> Sicilia, SARD--> Sardegna
# year: 2023
heat_cool_ninja = {}
for zone in zones:
    df_ninja = pd.read_csv(f"ninja_{zone}.csv", index_col=0, header=[3]).drop(columns=["total_demand","local_time"])/1e3  # MW
    heat_cool_ninja[zone] = df_ninja.sum(axis=1)

# Create dataframe with all ninja profiles
df_heat_cool_ninja = pd.DataFrame(heat_cool_ninja)
print("============> Electricity profile - ninja: heating + cooling (kW)")
display(df_heat_cool_ninja*1e3)

============> Electricity profile - ninja: heating + cooling (kW)


,CALA,CNOR,CSUD,NORD,SARD,SICI,SUD
time,,,,,,,
2023-01-01 00:00,0.424,0.782,0.141,1.314,0.468,0.261,0.000
2023-01-01 01:00,0.422,0.815,0.150,1.317,0.464,0.278,0.000
2023-01-01 02:00,0.414,0.838,0.158,1.305,0.455,0.292,0.000
2023-01-01 03:00,0.404,0.852,0.164,1.283,0.443,0.303,0.000
2023-01-01 04:00,0.396,0.871,0.171,1.270,0.434,0.315,0.000
...,...,...,...,...,...,...,...
2023-12-31 19:00,0.729,1.060,0.463,1.691,0.653,0.496,0.322
2023-12-31 20:00,0.728,1.017,0.454,1.652,0.613,0.480,0.317
2023-12-31 21:00,0.768,1.029,0.470,1.704,0.605,0.489,0.329


# 3 - Derive heating+cooling model profiles

In [201]:
# Create model profiles for all zones and years
heat_cool_model_profiles = {}
for year in [2019, 2040, 2050]:
    for zone in zones:
        # Get ninja demand
        heat_cool_ninja_demand = df_heat_cool_ninja[zone].sum()
        
        # Get model demand from df_demand
        heat_cool_model_demand = df_demand.loc[year, zone]
        
        # Scale ninja profile to match model demand
        heat_cool_model_profile = df_heat_cool_ninja[zone] * (heat_cool_model_demand / heat_cool_ninja_demand)
        
        # Store in dictionary with tuple key
        heat_cool_model_profiles[(year, zone)] = heat_cool_model_profile

# Create dataframe with multiindex columns
df_heat_cool_model = pd.DataFrame(heat_cool_model_profiles) # MW
df_heat_cool_model.columns = pd.MultiIndex.from_tuples(df_heat_cool_model.columns, names=['year', 'zone'])
print("============> Electricity profile - model: heating + cooling (MW)")
display(round(df_heat_cool_model,2))

============> Electricity profile - model: heating + cooling (MW)


year                2019                                                    \
zone                CALA    CNOR     CSUD     NORD    SARD    SICI     SUD   
time                                                                         
2023-01-01 00:00  117.63  488.46   307.46  5123.71  143.22  197.67    0.00   
2023-01-01 01:00  117.08  509.07   327.09  5135.41  141.99  210.55    0.00   
2023-01-01 02:00  114.86  523.44   344.54  5088.62  139.24  221.15    0.00   
2023-01-01 03:00  112.08  532.18   357.62  5002.83  135.57  229.48    0.00   
2023-01-01 04:00  109.86  544.05   372.88  4952.14  132.81  238.57    0.00   
...                  ...     ...      ...      ...     ...     ...     ...   
2023-12-31 19:00  202.25  662.11  1009.62  6593.75  199.83  375.65  413.77   
2023-12-31 20:00  201.97  635.25   989.99  6441.68  187.59  363.53  407.35   
2023-12-31 21:00  213.07  642.74  1024.88  6644.45  185.14  370.35  422.77   
2023-12-31 22:00  225.27  652.11  1064.13  6874.51  183.00  378.68  440.76   
2023-12-31 23:00  233.87  647.74  1081.58  6971.99  176.57  379.44  449.75   

year                2040                   ...                          \
zone                CALA    CNOR     CSUD  ...    SARD    SICI     SUD   
time                                       ...                           
2023-01-01 00:00  161.32  669.89   421.67  ...  196.41  271.09    0.00   
2023-01-01 01:00  160.56  698.16   448.58  ...  194.73  288.75    0.00   
2023-01-01 02:00  157.52  717.86   472.51  ...  190.96  303.29    0.00   
2023-01-01 03:00  153.71  729.85   490.45  ...  185.92  314.72    0.00   
2023-01-01 04:00  150.67  746.13   511.38  ...  182.14  327.18    0.00   
...                  ...     ...      ...  ...     ...     ...     ...   
2023-12-31 19:00  277.37  908.03  1384.62  ...  274.05  515.18  567.46   
2023-12-31 20:00  276.99  871.20  1357.71  ...  257.27  498.56  558.65   
2023-12-31 21:00  292.20  881.48  1405.55  ...  253.91  507.91  579.80   
2023-12-31 22:00  308.94  894.33  1459.38  ...  250.97  519.33  604.47   
2023-12-31 23:00  320.74  888.33  1483.31  ...  242.16  520.37  616.81   

year                2050                                                    
zone                CALA    CNOR     CSUD     NORD    SARD    SICI     SUD  
time                                                                        
2023-01-01 00:00  163.84  680.36   428.25  7136.60  199.48  275.33    0.00  
2023-01-01 01:00  163.07  709.07   455.59  7152.89  197.78  293.26    0.00  
2023-01-01 02:00  159.98  729.08   479.89  7087.71  193.94  308.03    0.00  
2023-01-01 03:00  156.11  741.26   498.11  6968.23  188.82  319.63    0.00  
2023-01-01 04:00  153.02  757.79   519.37  6897.62  184.99  332.29    0.00  
...                  ...     ...      ...      ...     ...     ...     ...  
2023-12-31 19:00  281.70  922.22  1406.25  9184.16  278.34  523.23  576.33  
2023-12-31 20:00  281.31  884.81  1378.92  8972.34  261.29  506.35  567.38  
2023-12-31 21:00  296.77  895.25  1427.52  9254.76  257.88  515.84  588.86  
2023-12-31 22:00  313.77  908.30  1482.19  9575.20  254.89  527.45  613.91  
2023-12-31 23:00  325.75  902.21  1506.48  9710.98  245.94  528.50  626.44  

[8760 rows x 21 columns]

# 4 - Create final profiles

In [221]:
# Create final profile dataframe
final_profiles = {}
for year in [2019, 2040, 2050]:
    for zone in zones:
        # Get components from df_profile_initial
        total_initial = df_profile_initial[(year, zone, 'total')]
        heating_initial = df_profile_initial[(year, zone, 'heating')]
        cooling_initial = df_profile_initial[(year, zone, 'cooling')]
        
        # Get ninja-based profile from df_heat_cool_model
        heat_cool_model = df_heat_cool_model[(year, zone)]
        
        # Reset index of heat_cool_model to match df_profile_initial
        heat_cool_model = heat_cool_model.reset_index(drop=True)
        heat_cool_model.index = total_initial.index
        
        # Calculate final profile: total - heating - cooling + heat_cool_model
        final_profile = total_initial - heating_initial - cooling_initial + heat_cool_model
        
        # Store in dictionary
        final_profiles[(year, zone)] = final_profile

# Create dataframe with multiindex columns
df_profile_final = pd.DataFrame(final_profiles)
df_profile_final.columns = pd.MultiIndex.from_tuples(df_profile_final.columns, names=['year', 'zone'])
print("============> Final electricity profile (MW)")
display(round(df_profile_final))
df_profile_final.to_csv("profile_electrification.csv")

============> Final electricity profile (MW)


year   2019                                                     2040          \
zone   CALA    CNOR    CSUD     NORD    SARD    SICI     SUD    CALA    CNOR   
hour                                                                           
1     597.0  2261.0  4461.0  15383.0   845.0  1803.0  1345.0   767.0  2912.0   
2     592.0  2212.0  4349.0  15091.0   811.0  1777.0  1188.0   761.0  2852.0   
3     540.0  2151.0  4133.0  14618.0   780.0  1679.0  1127.0   695.0  2776.0   
4     497.0  2075.0  3866.0  14176.0   756.0  1603.0  1104.0   641.0  2682.0   
5     486.0  2042.0  3741.0  13983.0   743.0  1563.0  1057.0   626.0  2641.0   
...     ...     ...     ...      ...     ...     ...     ...     ...     ...   
8756  885.0  3235.0  6431.0  21741.0  1252.0  2805.0  3198.0  1141.0  4163.0   
8757  823.0  3045.0  5937.0  20369.0  1175.0  2617.0  3097.0  1063.0  3920.0   
8758  762.0  2825.0  5619.0  19535.0  1093.0  2324.0  2879.0   986.0  3643.0   
8759  733.0  2695.0  5503.0  18957.0  1025.0  2190.0  2727.0   951.0  3478.0   
8760  715.0  2549.0  5230.0  18257.0   974.0  2045.0  2675.0   929.0  3294.0   

year          ...                            2050                           \
zone    CSUD  ...    SARD    SICI     SUD    CALA    CNOR    CSUD     NORD   
hour          ...                                                            
1     5677.0  ...  1084.0  2302.0  1702.0   891.0  3370.0  6731.0  22704.0   
2     5537.0  ...  1041.0  2270.0  1503.0   883.0  3293.0  6559.0  22260.0   
3     5265.0  ...  1002.0  2147.0  1426.0   805.0  3198.0  6228.0  21548.0   
4     4929.0  ...   970.0  2052.0  1397.0   741.0  3082.0  5823.0  20889.0   
5     4772.0  ...   955.0  2003.0  1337.0   723.0  3031.0  5630.0  20601.0   
...      ...  ...     ...     ...     ...     ...     ...     ...      ...   
8756  8243.0  ...  1605.0  3589.0  4090.0  1318.0  4827.0  9633.0  32169.0   
8757  7616.0  ...  1506.0  3349.0  3961.0  1224.0  4541.0  8885.0  30106.0   
8758  7218.0  ...  1403.0  2980.0  3687.0  1129.0  4207.0  8399.0  28815.0   
8759  7076.0  ...  1316.0  2811.0  3496.0  1084.0  4008.0  8219.0  27909.0   
8760  6732.0  ...  1251.0  2628.0  3431.0  1055.0  3788.0  7802.0  26835.0   

year                          
zone    SARD    SICI     SUD  
hour                          
1     1264.0  2712.0  2041.0  
2     1213.0  2670.0  1802.0  
3     1166.0  2520.0  1711.0  
4     1130.0  2404.0  1676.0  
5     1112.0  2342.0  1604.0  
...      ...     ...     ...  
8756  1874.0  4210.0  4802.0  
8757  1759.0  3925.0  4648.0  
8758  1636.0  3481.0  4316.0  
8759  1532.0  3276.0  4083.0  
8760  1456.0  3056.0  4002.0  

[8760 rows x 21 columns]

# 5 - Check

In [219]:
total_demand_2019_ex_post = df_profile_final[2019].sum().sum()/1e6
total_demand_2040_ex_post = df_profile_final[2040].sum().sum()/1e6
total_demand_2050_ex_post = df_profile_final[2050].sum().sum()/1e6

increase_2019_2040_ex_post = (total_demand_2040_ex_post/total_demand_2019_ex_post) * 100
increase_2019_2050_ex_post = (total_demand_2050_ex_post/total_demand_2019_ex_post) * 100

print(increase_2019_2040_ex_post - (100 + data[2040]['increase'])) # should be 0
print(increase_2019_2050_ex_post - (100 + data[2050]['increase'])) # should be 0

6.42359054836561e-10
-1.6058834262366872e-09
